In [1]:
import sqlite3, time
from ib_insync import * 
import pandas as pd, numpy as np 
from random import randint
from datetime import * 

util.startLoop()        # use this when working in notebooks 

import sys 
sys.path.insert(1, "src")           # add directory to python path 
sys.path.insert(1, "database")      # add directory to python path 

ib = IB() 
ib.connect('127.0.0.1', 7497, clientId=randint(0, 9999), timeout=0) 

<IB connected to 127.0.0.1:7497 clientId=8341>

In [12]:
# Get all positions
print(ib.positions())

[Position(account='DU2804404', contract=Stock(conId=517593749, symbol='IONQ', exchange='NYSE', currency='USD', localSymbol='IONQ', tradingClass='IONQ'), position=-4000.0, avgCost=5.0897533)]


In [19]:
for elem in ib.reqAllOpenOrders(): 
    # ib.cancelOrder(elem)
    print(elem)

Order(orderId=8, clientId=8341, permId=1009298699, action='BUY', totalQuantity=1000.0, orderType='LMT', lmtPrice=5.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, trailStopPrice=6.0, openClose='', volatilityType=0, deltaNeutralOrderType='None', deltaNeutralOpenClose='?', referencePriceType=0, account='DU2804404', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True)
Order(orderId=9, clientId=8341, permId=1009298700, action='BUY', totalQuantity=1000.0, orderType='LMT', lmtPrice=5.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, trailStopPrice=6.0, openClose='', volatilityType=0, deltaNeutralOrderType='None', deltaNeutralOpenClose='?', referencePriceType=0, account='DU2804404', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True)
Order(orderId=10, clientId=8341, permId=1009298701, action='BUY', totalQuantity=1000.0, orderType='LMT', lmtPrice=5.0, auxPrice=0.0, tif='GTC', ocaType=3, dis

Peer closed connection.


In [4]:
ionq = Stock("IONQ", "SMART", "USD", primaryExchange="NASDAQ")

In [6]:
# Limit Order filled in and out of RTH
# Good till canceled orders must be written with tif='GTC' (time in force)
order1 = LimitOrder('BUY', 10, 4.95, tif='GTC')   
order2 = LimitOrder('SELL', 10, 5.20)   
order3 = LimitOrder('BUY', 10, 4.95, outsideRth=True) 
order4 = LimitOrder('SELL', 10, 4.8, outsideRth=True) 

# Market Orders. They cannot be filled outside RTH
order5 = MarketOrder('BUY', 10) 
order6 = MarketOrder('SELL', 10) 

# Stop loss orders filled in and out of RTH 
order7 = StopOrder('SELL', 10, 4.50) 
order8 = StopOrder('SELL', 10, 4.50, outsideRth=True) 

# Placing the orders 
# ib.placeOrder(stock, order1)

# Canceling a limit/market/stop order 
# ib.cancelOrder(order1) 

# A bracket order that submits a limit order to buy (parent order), 
# a limit order to sell, and a stop loss at the same time (child orders)
# The transmit attribute is set to false for the 1st 2 orders so that the final one, which is set to true, 
# can trigger all 3 of them to be sent to IBKR servers at once. Removes risk of accidental execution. 
order9 = ib.bracketOrder("BUY", 1000, 4.5, 4.8, 4.3) 
for ord in order9: 
    # ib.placeOrder(ionq, ord) 
    # ib.sleep(1)     # important for child orders to get transmitted
    print(ord)
    pass 

LimitOrder(orderId=3, action='BUY', totalQuantity=1000, lmtPrice=4.5, transmit=False)
LimitOrder(orderId=4, action='SELL', totalQuantity=1000, lmtPrice=4.8, transmit=False, parentId=3)
StopOrder(orderId=5, action='SELL', totalQuantity=1000, auxPrice=4.3, parentId=3)


In [20]:
short_sell_1 = LimitOrder("SELL", 1000, 5.13) 
short_trade_1 = ib.placeOrder(ionq, short_sell_1) 

In [16]:
short_buy_1 = LimitOrder("BUY", 1000, 5.00, tif='GTC') 
ib.placeOrder(ionq, short_buy_1)  

Trade(contract=Stock(symbol='IONQ', exchange='SMART', primaryExchange='NASDAQ', currency='USD'), order=LimitOrder(orderId=10, clientId=8341, action='BUY', totalQuantity=1000, lmtPrice=5.0, tif='GTC'), orderStatus=OrderStatus(orderId=10, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2022, 10, 14, 2, 0, 29, 798752, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

In [8]:
custom_order = LimitOrder("BUY", 10, 4.00, orderId=1, discretionaryAmount=0.00) 
print(custom_order.orderId)

1
